In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
data = pd.read_csv("breast_cancer/data.csv")

In [14]:
data.sample(20)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
280,8912049,M,19.16,26.60,126.20,1138.0,0.10200,0.14530,0.19210,0.096640,...,35.90,159.80,1724.0,0.17820,0.38410,0.57540,0.18720,0.3258,0.09720,NaN
173,871641,B,11.08,14.71,70.21,372.7,0.10060,0.05743,0.02363,0.025830,...,16.82,72.01,396.5,0.12160,0.08240,0.03938,0.04306,0.1902,0.07313,NaN
10,845636,M,16.02,23.24,102.70,797.8,0.08206,0.06669,0.03299,0.033230,...,33.88,123.80,1150.0,0.11810,0.15510,0.14590,0.09975,0.2948,0.08452,NaN
76,8610629,B,13.53,10.94,87.91,559.2,0.12910,0.10470,0.06877,0.065560,...,12.49,91.36,605.5,0.14510,0.13790,0.08539,0.07407,0.2710,0.07191,NaN
480,912193,B,12.16,18.03,78.29,455.3,0.09087,0.07838,0.02916,0.015270,...,27.87,88.83,547.4,0.12080,0.22790,0.16200,0.05690,0.2406,0.07729,NaN
249,884689,B,11.52,14.93,73.87,406.3,0.10130,0.07808,0.04328,0.029290,...,21.19,80.88,491.8,0.13890,0.15820,0.18040,0.09608,0.2664,0.07809,NaN
450,9111596,B,11.87,21.54,76.83,432.0,0.06613,0.10640,0.08777,0.023860,...,28.18,83.51,507.2,0.09457,0.33990,0.32180,0.08750,0.2305,0.09952,NaN
402,904689,B,12.96,18.29,84.18,525.2,0.07351,0.07899,0.04057,0.018830,...,24.61,96.31,621.9,0.09329,0.23180,0.16040,0.06608,0.3207,0.07247,NaN
18,849014,M,19.81,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.094980,...,30.88,186.80,2398.0,0.15120,0.31500,0.53720,0.23880,0.2768,0.07615,NaN
38,855133,M,14.99,25.20,95.54,698.8,0.09387,0.05131,0.02398,0.028990,...,25.20,95.54,698.8,0.09387,0.05131,0.02398,0.02899,0.1565,0.05504,NaN


In [35]:
# Each row of the dataset is long. We choose a smaller size.
train = data.iloc[0:int(569/10), :]
test = data.iloc[int(569/10*5):int(569/10*7), :]

In [16]:
train.to_csv("breast_cancer/bd_train.txt", index = False)

In [36]:
# gpt 3.5 with hint
def classify_row(row):
    if row['smoothness_mean'] <= 0.1 and row['compactness_mean'] <= 0.2:
        return 'B'  # Benign
    else:
        return 'M'  # Malignant

In [37]:
test_pred = test.apply(classify_row, axis = 1)
np.mean(test_pred == test["diagnosis"])

0.7280701754385965

In [33]:
# after feedback
def classify_row_m(row):
    if row['smoothness_mean'] <= 0.1 and row['compactness_mean'] <= 0.2:
        return 'B'  # Benign
    elif row['smoothness_mean'] <= 0.11 and row['compactness_mean'] <= 0.3:
        return 'B'  # Benign (modified rule)
    else:
        return 'M'  # Malignant

In [38]:
test_pred = test.apply(classify_row_m, axis = 1)
np.mean(test_pred == test["diagnosis"])

0.8070175438596491

In [39]:
# claude 2

def predict(row):
    if row['radius_worst'] > 16:
        return 'M'
    elif row['perimeter_worst'] > 107:
        return 'M'
    elif row['area_worst'] > 569:
        return 'M'
    elif row['smoothness_worst'] > 0.15:
        return 'M'
    elif row['compactness_worst'] > 0.38:
        return 'M' 
    elif row['concavity_worst'] > 0.43:
        return 'M'
    elif row['concave points_worst'] > 0.13:
        return 'M'
    else:
        return 'B'

In [40]:
test_pred = test.apply(predict, axis = 1)
np.mean(test_pred == test["diagnosis"])

0.6052631578947368

In [41]:
# after feedback
def predict_m(row):
    if (row['radius_worst'] > 16) and (row['texture_worst'] > 21):
        return 'M'
    elif (row['perimeter_worst'] > 115) and (row['area_worst'] > 760):
        return 'M'
    elif (row['smoothness_worst'] > 0.13) and (row['compactness_worst'] > 0.25): 
        return 'M'
    elif (row['concavity_worst'] > 0.3) and (row['concave points_worst'] > 0.1):
        return 'M'
    else:
        return 'B'

In [42]:
test_pred = test.apply(predict_m, axis = 1)
np.mean(test_pred == test["diagnosis"])

0.8859649122807017

In [ ]:
# Accuracy by GPT 4
# with context & before feedback: 85%
# with context & after feedback: 92.4%